# NBA MVP Prediction – End-to-End Machine Learning Projekt
### Einleitung
Ziel dieses Projekts ist es, ein Machine-Learning-Modell zu entwickeln, das vorhersagen kann, ob ein NBA-Spieler MVP wird. Dabei handelt es sich um ein Klassifikationsproblem. Die Vorhersage basiert auf Spielerstatistiken sowie Teamleistungen. Das Projekt wird vollständig dokumentiert, inklusive Datenbeschaffung, Aufbereitung, Modelltraining und Deployment.

Verwendete Hauptbibliotheken:
pandas: Datenimport, -bereinigung und -analyse

numpy: Numerische Berechnungen

scikit-learn: Machine Learning Modelle, Evaluation, Datenaufteilung

matplotlib & seaborn: Visualisierung

jupyter: Interaktive Entwicklung und Präsentation im Notebook

huggingface_hub: Deployment auf Hugging Face


In [28]:
# Datenmanipulation
import pandas as pd
import numpy as np

# Visualisierung
import matplotlib.pyplot as plt
import seaborn as sns

# XGBoost
import xgboost as xgb
from xgboost import XGBClassifier

# Machine Learning – Modelle & Tools
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, roc_curve

# Optional – Web Scraping & API
import requests
from bs4 import BeautifulSoup

# Hugging Face Deployment
from huggingface_hub import InferenceClient, login

# Jupyter spezifisch (für Plot-Darstellung inline)
%matplotlib inline


# 1. Datenakquise und Datenstruktur

In diesem Projekt wurden die benötigten Daten aus drei verschiedenen Quellen manuell beschafft und kombiniert. Ziel war es, aus öffentlich zugänglichen Statistiken ein konsistentes Machine-Learning-Datenset für die NBA-Saisons von **2004/05 bis 2023/24** zu erstellen.

### Quellen:
- [Basketball Reference](https://www.basketball-reference.com/)
- [Stathead.com](https://www.stathead.com/)

### Verwendete Datensätze:

1. **MVP-Daten**  
   Für jede Saison wurde der Most Valuable Player (MVP) manuell von *Basketball Reference* entnommen. Erfasst wurden:
   - Spielername  
   - Team  
   - Saison

2. **Team-Winrate (Regular Season Standings)**  
   Über *Stathead.com* wurden für jede Saison die regulären Saisonplatzierungen extrahiert. Verwendet wurden:
   - Teamname  
   - Winrate (Siege / Spiele)

3. **Spielerstatistiken**  
   Ebenfalls über *Stathead.com* wurden für jede Saison die Top 200 Spieler nach Punkten pro Spiel (PPG) ausgewählt. Zusätzlich wurden folgende Werte berechnet:
   - **RPG (Rebounds pro Spiel)** = Total Rebounds / Games Played  
   - **APG (Assists pro Spiel)** = Total Assists / Games Played  
   Erfasst wurden:
   - Spielername  
   - Team  
   - Saison  
   - Punkte pro Spiel (PPG)  
   - Rebounds pro Spiel (RPG)  
   - Assists pro Spiel (APG)

### Zusammenführung der Daten

Die drei Datensätze wurden pro Saison anhand von Spielername, Team und Saison zusammengeführt. Die finale Datenstruktur eines kombinierten Eintrags sieht folgendermaßen aus:

Season | Player | Team | Winrate | PPG | RPG | APG | MVP

- Die Spalte **MVP** ist die Zielvariable und nimmt den Wert **1** für den MVP der Saison an, sonst **0**.
- Danach wurden alle Saisons von 2004/05 bis 2023/24 zu einem Gesamtdatensatz zusammengeführt, der als Grundlage für Feature Engineering, Modelltraining und Evaluation dient.

### Tools:
- Python (für Verarbeitung, Analyse und Modellierung)
- Manuelles CSV-Scraping oder Exportfunktionen von Stathead.com und Basketball Reference



## MVP Daten (04/05 - 23/24)

Dieses Skript verarbeitet die MVP-Voting-Daten der Saisons 2004/05 bis 2023/24.
Ziel ist es, aus den rohen CSV-Dateien pro Saison einheitliche, saubere Daten im Format
Player, Team, PPG, RPG, APG, Season, MVP (1/0) zu extrahieren.
Die bereinigten Daten werden in einer CSV-Datei gespeichert und sind bereit für das spätere Mergin

In [29]:
# 📓 Notebook-Sektion: MVP-Daten einlesen und vorbereiten

import pandas as pd
import glob
import os

# 1. Pfad zur MVP-Datenquelle definieren
base_path = "Data_dirty/mvp"

# 2. Alle CSV-Dateien im MVP-Ordner finden
mvp_files = glob.glob(os.path.join(base_path, "*.csv"))

# 3. Vorbereitung: Liste zum Speichern der sauberen DataFrames
mvp_dfs = []

# 4. Jede MVP-Datei einlesen, bereinigen & in Zielstruktur bringen
for file in mvp_files:
    try:
        #Saison aus Dateiname extrahieren (z. B. "mvp_04_05.csv")
        season = os.path.splitext(os.path.basename(file))[0].split("_")[-2] + "/" + os.path.splitext(os.path.basename(file))[0].split("_")[-1]

        #CSV ohne Header einlesen, da keine echten Spaltennamen vorhanden
        df = pd.read_csv(file, sep=";", encoding="utf-8", header=None)

        #Saison als neue Spalte ergänzen
        df["Season"] = season

        #Spalte mit -9999 entfernen, falls vorhanden
        df = df.loc[:, ~df.columns.astype(str).str.contains("-9999", na=False)]

        #MVP_Label setzen: Wenn erste Spalte == 1 → MVP
        df["MVP"] = df[0].apply(lambda x: 1 if str(x).strip() == "1" else 0)

        #Zielstruktur extrahieren (Spaltenpositionen manuell geprüft)
        df_clean = pd.DataFrame({
            "Player": df[1],
            "Team": df[3],
            "PPG": df[10],
            "RPG": df[11],
            "APG": df[12],
            "Season": df["Season"],
            "MVP": df["MVP"]
        })

        #bereinigte Version speichern
        mvp_dfs.append(df_clean)

    except Exception as e:
        print(f"❌ Fehler beim Verarbeiten von {file}: {e}")

# Ergebnis: Alle Saisons in ein DataFrame zusammenführen
if mvp_dfs:
    df_mvp_final = pd.concat(mvp_dfs, ignore_index=True)
    display(df_mvp_final.head())
    print(f"\n✅ {len(df_mvp_final)} Zeilen finaler MVP-Datensatz bereit.")

    # Finalen MVP-Datensatz speichern
    os.makedirs("data_clean", exist_ok=True)
    df_mvp_final.to_csv("data_clean/mvp_final.csv", index=False)
    print("💾 Datei gespeichert unter: data_clean/mvp_final.csv")
else:
    print("❌ Keine gültigen MVP-Daten geladen.")


❌ Keine gültigen MVP-Daten geladen.


## Player Stats (04/05 - 23/24)

Dieses Notebook-Skript verarbeitet die Spielerstatistiken der Saisons 2004/05 bis 2023/24.
Da die Dateien keine echten Header-Zeilen enthalten, werden feste Spaltenpositionen verwendet.
Es werden berechnet:
- APG = AST / G
- RPG = TRB / G
- PPG wird direkt aus der Datei entnommen (zuvor berechnet).
Zielspalten: Player, Team, PPG, RPG, APG, Season
Hinweis: Nicht-numerische Werte wie 'G' oder Spaltenköpfe in Datenzeilen werden übersprungen.

In [30]:
# 1. Pfad zum Player-Stats-Ordner definieren
base_path = "data_dirty/player_stats"
player_files = glob.glob(os.path.join(base_path, "*.csv"))

# 2. Liste zum Sammeln der bereinigten DataFrames
player_dfs = []

# 3. Jede Datei laden, berechnen und vorbereiten
for file in player_files:
    try:
        # Saison extrahieren aus Dateinamen
        season = os.path.splitext(os.path.basename(file))[0].split("_")[-2] + "/" + os.path.splitext(os.path.basename(file))[0].split("_")[-1]

        # CSV einlesen – ohne Header, weil Daten direkt ab erster Zeile
        df = pd.read_csv(file, sep=";", encoding="utf-8", header=None)

        # Nur Zeilen behalten, die numerisch sind (z. B. nicht "G" oder "Player")
        df = df[df[6].apply(lambda x: str(x).replace('.', '', 1).isdigit())]

        # Zielstruktur über feste Spaltenpositionen extrahieren
        df_clean = pd.DataFrame({
            "Player": df[1],
            "Team": df[5],
            "G": df[6].astype(float),
            "PPG": df[2].astype(float),
            "TRB": df[20].astype(float),
            "AST": df[21].astype(float),
            "Season": season
        })

        # Statistiken berechnen und runden
        df_clean["RPG"] = (df_clean["TRB"] / df_clean["G"]).round(1)
        df_clean["APG"] = (df_clean["AST"] / df_clean["G"]).round(1)

        # Finale Spaltenauswahl
        df_final = df_clean[["Player", "Team", "Season", "PPG", "RPG", "APG"]]

        player_dfs.append(df_final)

    except Exception as e:
        print(f"Fehler in Datei {file}: {e}")

# 4. Gesamten Player-Stats-Datensatz zusammenfügen und speichern
if player_dfs:
    df_stats_final = pd.concat(player_dfs, ignore_index=True)
    os.makedirs("data_clean", exist_ok=True)
    df_stats_final.to_csv("data_clean/player_stats_final.csv", index=False)
    print("Spielerstatistik gespeichert unter: data_clean/player_stats_final.csv")
else:
    print("Keine gültigen Player-Statistik-Dateien gefunden.")


Spielerstatistik gespeichert unter: data_clean/player_stats_final.csv


## Team Stats (04/05 - 23/24)
Dieses Notebook-Skript verarbeitet die Team-Siegquoten von 2004/05 bis 2023/24.
Ziel ist es, aus jeder CSV-Datei die Teamnamen und die Win Percentage (W/L%) auszulesen
und mit der jeweiligen Saison in einem einheitlichen DataFrame zu speichern.

In [31]:
# 1. Pfad zum Team-Winrate-Ordner definieren
base_path = "data_dirty/team_winrates"
team_files = glob.glob(os.path.join(base_path, "*.csv"))

# 2. Liste zum Sammeln der bereinigten DataFrames
team_dfs = []

# 3. Jede Datei einlesen und vorbereiten
for file in team_files:
    try:
        # Saison extrahieren
        season = os.path.splitext(os.path.basename(file))[0].split("_")[-2] + "/" + os.path.splitext(os.path.basename(file))[0].split("_")[-1]

        # CSV einlesen – mit Header ab Zeile 1
        df = pd.read_csv(file, sep=";", encoding="utf-8", header=0)

        # Nur Teams und Win Percentage extrahieren
        df_clean = pd.DataFrame({
            "Team": df["Team"],
            "Win%": df["W/L%"],
            "Season": season
        })

        team_dfs.append(df_clean)

    except Exception as e:
        print(f"Fehler in Datei {file}: {e}")

# 4. Gesamten Team-Winrate-Datensatz zusammenfügen und speichern
if team_dfs:
    df_teams_final = pd.concat(team_dfs, ignore_index=True)
    os.makedirs("data_clean", exist_ok=True)
    df_teams_final.to_csv("data_clean/team_winrate_final.csv", index=False)
    print("Team-Winrate-Daten gespeichert unter: data_clean/team_winrate_final.csv")
else:
    print("Keine gültigen Team-Winrate-Dateien gefunden.")

Team-Winrate-Daten gespeichert unter: data_clean/team_winrate_final.csv


# 2. Datenbereinigung (Data Cleaning)


## Pre-Merge Checks
In diesem Abschnitt bereinigen wir die drei vorbereiteten Datensätze
(MVP, Spielerstatistiken, Team-Winrate) und führen wichtige Pre-Merge-Checks durch:
- Einheitliche Schreibweise für Schlüsselspalten
- Duplikatprüfung
- Prüfung auf fehlende Werte (NaNs)
- Prüfung auf Spieler mit mehreren Teams in einer Saison

In [ ]:
# CSV-Dateien laden
mvp = pd.read_csv("data_clean/mvp_final.csv")
stats = pd.read_csv("data_clean/player_stats_final.csv")
teams = pd.read_csv("data_clean/team_winrate_final.csv")


Spalten in MVP: ['Player', 'Team', 'PPG', 'RPG', 'APG', 'Season', 'MVP']
Spalten in Stats: ['Rk;Player;PTS/G;Season;Age;Team;G;GS;AS;MP;FG;FGA;2P;2PA;3P;3PA;FT;FTA;ORB;DRB;TRB;AST;STL;BLK;TOV;PF;PTS;FG%;2P%;3P%;FT%;TS%;eFG%;Pos;Player-additional']
Spalten in Teams: ['Team', 'Win%', 'Season']


In [55]:
if df_stats.columns.size == 1 and ";" in df_stats.columns[0]:
    df_stats = pd.read_csv("data_clean/player_stats_final.csv", sep=",", encoding="utf-8")

In [57]:
# Spalten prüfen
print("Spalten in MVP:", df_mvp.columns.tolist())
print("Spalten in Stats:", df_stats.columns.tolist())
print("Spalten in Teams:", df_team.columns.tolist())


Spalten in MVP: ['Player', 'Team', 'PPG', 'RPG', 'APG', 'Season', 'MVP']
Spalten in Stats: ['Player', 'Team', 'Season', 'PPG', 'RPG', 'APG']
Spalten in Teams: ['Team', 'Win%', 'Season']


In [58]:
# 3. Schreibweise vereinheitlichen
for df in [df_mvp, df_stats, df_team]:
    if "Player" in df.columns:
        df["Player"] = df["Player"].astype(str).str.strip()
    if "Team" in df.columns:
        df["Team"] = df["Team"].astype(str).str.upper().str.strip()
    if "Season" in df.columns:
        df["Season"] = df["Season"].astype(str).str.strip()


In [59]:
# 4. NaN-Prüfung
print("\nFehlende Werte je Spalte:")
print("MVP:", df_mvp.isna().sum())
print("Stats:", df_stats.isna().sum())
print("Teams:", df_team.isna().sum())


Fehlende Werte je Spalte:
MVP: Player    0
Team      0
PPG       0
RPG       0
APG       0
Season    0
MVP       0
dtype: int64
Stats: Player    0
Team      0
Season    0
PPG       0
RPG       0
APG       0
dtype: int64
Teams: Team      0
Win%      0
Season    0
dtype: int64


In [60]:

# 5. Duplikate prüfen (Player + Season)
print("\nDuplikate:")
if "Player" in df_mvp.columns and "Season" in df_mvp.columns:
    print("MVP:", df_mvp.duplicated(subset=["Player", "Season"]).sum())
if "Player" in df_stats.columns and "Season" in df_stats.columns:
    print("Stats:", df_stats.duplicated(subset=["Player", "Season"]).sum())




Duplikate:
MVP: 0
Stats: 0


In [61]:
# 6. Spieler mit mehreren Teams pro Saison prüfen
if "Player" in df_stats.columns and "Season" in df_stats.columns and "Team" in df_stats.columns:
    mehrfach_teams = df_stats.groupby(["Player", "Season"])["Team"].nunique().reset_index()
    mehrfach_teams = mehrfach_teams[mehrfach_teams["Team"] > 1]
    print("\nSpieler mit mehreren Teams in einer Saison:")
    print(mehrfach_teams)
    print("\nAnzahl betroffener Spieler:", mehrfach_teams.shape[0])


Spieler mit mehreren Teams in einer Saison:
Empty DataFrame
Columns: [Player, Season, Team]
Index: []

Anzahl betroffener Spieler: 0


## 3. Datenzusammenführung (Data Merging)
In diesem Abschnitt führen wir die drei bereinigten Datensätze zusammen:
1. MVP-Daten (inkl. MVP-Label)
2. Spielerstatistiken (ausgeschlossen: Spieler mit mehreren Teams pro Saison)
3. Team-Winrates

Ziel ist ein einheitlicher, vollständiger Datensatz mit folgenden Spalten:
Player, Team, Season, PPG, RPG, APG, Win%, MVP


In [76]:
# 1. Spieler mit mehreren Teams entfernen

# Hinweis: Spieler mit Teamkürzel länger als 3 Buchstaben entfernen,
# da dies meist zusammengesetzte Teams sind (bei Teamwechsel in einer Saison)
df_stats = df_stats[df_stats["Team"].str.len() == 3]

# Team-Kürzel-Anpassungen: historische Namensänderungen berücksichtigen
def vereinheitliche_teamnamen(team):
    if team == "SEA":
        return "OKC"
    if team == "CHA":
        return "CHO"
    return team

In [77]:
# Anwenden auf alle relevanten DataFrames
df_mvp["Team"] = df_mvp["Team"].apply(vereinheitliche_teamnamen)
df_stats["Team"] = df_stats["Team"].apply(vereinheitliche_teamnamen)
df_team["Team"] = df_team["Team"].apply(vereinheitliche_teamnamen)
mehrfach = df_stats.groupby(["Player", "Season"])["Team"].nunique().reset_index()
mehrfach = mehrfach[mehrfach["Team"] > 1]
df_stats = df_stats.merge(mehrfach[["Player", "Season"]], on=["Player", "Season"], how="left", indicator=True)
df_stats = df_stats[df_stats["_merge"] == "left_only"].drop(columns=["_merge"])

In [78]:
#  MVP-Daten mit Team-Winrate verbinden
df_mvp_merged = df_mvp.merge(df_team, on=["Team", "Season"], how="left")

In [79]:
# Spieler aus df_stats, die nicht in df_mvp sind (Rest der Liga)
df_keys = df_mvp[["Player", "Season"]].drop_duplicates()
df_rest = df_stats.merge(df_keys, on=["Player", "Season"], how="left", indicator=True)
df_rest = df_rest[df_rest["_merge"] == "left_only"].drop(columns=["_merge"])
df_rest["MVP"] = 0

In [80]:
# Restliche Spieler mit Team-Winrate verbinden
df_rest = df_rest.merge(df_team, on=["Team", "Season"], how="left")

In [81]:
# Beide DataFrames zusammenführen
df_mvp_final = df_mvp_merged[["Player", "Team", "Season", "PPG", "RPG", "APG", "Win%", "MVP"]]
df_rest_final = df_rest[["Player", "Team", "Season", "PPG", "RPG", "APG", "Win%", "MVP"]]
df_final = pd.concat([df_mvp_final, df_rest_final], ignore_index=True)

In [82]:
# Exportieren
df_final.to_csv("data_clean/final_dataset.csv", index=False)
print("Finaler Datensatz gespeichert unter: data_clean/final_dataset.csv")

Finaler Datensatz gespeichert unter: data_clean/final_dataset.csv


In [84]:
# Anzahl MVPs im finalen Datensatz zählen
anzahl_mvps = df_final[df_final["MVP"] == 1].shape[0]
print(f"Anzahl MVPs im final_dataset.csv: {anzahl_mvps}")

# MVPs pro Saison
print("\nMVPs pro Saison:")
print(df_final[df_final["MVP"] == 1].groupby("Season").size())

Anzahl MVPs im final_dataset.csv: 20

MVPs pro Saison:
Season
04/05    1
05/06    1
06/07    1
07/08    1
08/09    1
09/10    1
10/11    1
11/12    1
12/13    1
13/14    1
14/15    1
15/16    1
16/17    1
17/18    1
18/19    1
19/20    1
20/21    1
21/22    1
22/23    1
23/24    1
dtype: int64


## 4. Feature Engineering


## 5. Explorative Datenanalyse (EDA)


## 6. Modelltraining


## 7. Modellbewertung (Model Evaluation)


## 8. Deployment


## 9. Fazit und Ausblick


## 10. Anhang
